In [ ]:
import numpy as np

In [ ]:
class Pipe(object):
    '''
    Args:
        Ro (np.array or float): Outer radius of pipe
        Ri (np.array or float): Inner radius of pipe
        Rb (np.array or float): Effective bit radius contacting rock (m)
        alpha (np.array or float): Axial velocity of the drill stem.
        rho (np.array or float): Density of the drill stem.
    '''
    def __init__(self, 
             Ro=.1365, 
             Ri=.0687, 
             Rb=.16,
             alpha=4875,
             rho=7800
                ):
        self.Ro = Ro
        self.Ri = Ri
        self.Rb = Rb
        self.alpha = alpha
        self.rho = rho
        
    @property
    def A1(self):
        '''
        Effective drill stem area.
        '''
        return np.pi*((self.Ro**2)-(self.Ri**2))
    
    @property
    def Ab(self):
        '''
        Area of the bit contacting rock.
        '''
        return np.pi * (self.Rb**2)
        
    @property
    def Z1(self):
        '''
        Steel impedance.
        '''
        return self.Ab * self.rho * self.alpha * 0.00001

class Rock(object):
    '''
    Args:
        alpha (np.array or float): Velocity of the rock.
        rho (np.array or float): Density of the rock.
        modulus (np.array or float): Modulus of the rock.
    '''
    def __init__(self, alpha=None, rho=None, modulus=None):
        self.alpha = alpha
        self.rho = rho
        self.modulus = modulus
        
#     @property
#     def modulus(self):
#         if self._modulus:
#             return self._modulus
#         return self._rho * self._alpha
    
#     @property
#     def rho(self):
#         if self._rho:
#             return self._rho
#         return self._rho * self._alpha

In [ ]:
alpha=np.arange(1000, 3500 + 25, 25)
rho=np.arange(1300, 2500 + 25, 25)

In [ ]:
pipe = Pipe(Ro=.1365, Ri=.0687, Rb=.16, alpha=4875, rho=7800)
rock = Rock(alpha, rho)

In [ ]:
from scipy import signal

In [ ]:
def lowpass_filter(s, cutoff=30, sampling_frequency=0.3):
    fc = cutoff # Cut-off frequency of the filter
    fs = sampling_frequency
#     w = fc / (fs / 2) # Normalize the frequency
    w = 0.02
    b, a = signal.butter(5, 0.05, 'low')
    return signal.filtfilt(b, a, s)

def amplitude_and_phase(pipe, rock, f):
    '''
    From a given pipe and rock instances and a range of frequencies compute amplitude and phase.
    '''
    k = 2 * np.pi * f / rock.alpha
    cot_phi = -1 * (k * pipe.Rb * (1+6*np.sqrt(3))/12)
    
    Zb = ((pipe.Ab * rock.rho * rock.alpha) / (k * pipe.Rb)) / (1j - cot_phi) * .00001
    
    RC_complex = (pipe.Z1 - Zb) / (pipe.Z1 + Zb)

    RC = np.abs(RC_complex)
    
    RC_tanphi = RC_complex.imag / RC_complex.real
    
    phi = np.arctan(RC_tanphi)
    return RC_complex, RC, phi

def inverse_transform(amplitude, phase):
    freq_domain = np.nan_to_num(amplitude * np.cos(phase) + (amplitude * np.sin(phase))* 1j)
    freq_domain_real = freq_domain.real
    freq_domain_imag = freq_domain.imag
    inverse = np.fft.ifft(freq_domain_real + freq_domain_imag*1j)
    return np.fft.fftshift(inverse)

In [ ]:
from obspy import read, Trace, Stream, UTCDateTime
from obspy.core import AttribDict
from obspy.io.segy.segy import SEGYTraceHeader, SEGYBinaryFileHeader
from obspy.io.segy.core import _read_segy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
frequency_resolution = 0.05
N = 100000

In [ ]:
f = np.fft.fftfreq(N, frequency_resolution)

In [ ]:
samples_from_center = 100

In [ ]:
import seaborn as sns

In [ ]:
r, a = 1700, 1700

RC_complex, amp, phase = amplitude_and_phase(pipe, Rock(r, a), f)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1)

freq_domain = amp * np.cos(phase) + (amp * np.sin(phase))* 1j

# huge_step = np.argmax(np.diff(freq_domain)[1:])+1

# freq_domain.real[huge_step:] = (0-freq_domain.real[huge_step:])
# freq_domain.imag[huge_step:] = freq_domain.imag.max() - (freq_domain.imag[huge_step:]) - freq_domain.imag.max()

# freq_domain_real = np.r_[freq_domain.real, freq_domain.real[::-1]][:-1]
# freq_domain_imag = np.r_[freq_domain.imag, freq_domain.imag[::-1]][:-1]

# freq_domain_imag[int(len(freq_domain_imag)/2):] = - freq_domain_imag[int(len(freq_domain_imag)/2):]

# freq_domain_real = freq_domain.real
# freq_domain_imag = freq_domain.imag

inverse = np.fft.ifft(freq_domain_real + freq_domain_imag*1j)

ax1.plot(f, amp)
ax2.plot(f, phase)

ax3.plot(np.fft.fftshift(inverse.real)[(int(N/2))-samples_from_center:(int(N/2))+samples_from_center])
ax3.plot(np.fft.fftshift(inverse.imag)[(int(N/2))-samples_from_center:(int(N/2))+samples_from_center])


ax1.set_ylabel('amplitude')
ax2.set_ylabel('phase')
ax3.set_ylabel('')
ax3.set_xlabel('')
# ax3.set_ylim(0)
for ax in [ax1, ax2, ax3]:
    ax.grid()
fig.suptitle('rho={}, alpha={}'.format(r, a))
fig = plt.gcf()
fig.set_size_inches(15,7.5)
fig.dpi = 200

In [ ]:
f

In [ ]:
((pipe.Ab * r * a) / (k * pipe.Rb))

In [ ]:
(k * pipe.Rb) / (1j - cot_phi)

In [ ]:
RC_complex[1]

In [ ]:
k = 2 * np.pi * f / a
cot_phi = -1 * (k * pipe.Rb * (1+6*np.sqrt(3))/12)

Zb = ((pipe.Ab * r * a) / (k * pipe.Rb)) / (1j - cot_phi) * .00001

RC_complex = (pipe.Z1 - Zb) / (pipe.Z1 + Zb)
RC_complex[0] = -1+0j
RC = np.abs(RC_complex)

RC_tanphi = RC_complex.imag / RC_complex.real

phi = np.arctan(RC_tanphi)

In [ ]:
plt.plot(RC_complex)

In [ ]:
print('3 first real for freq domain: ', (amp[:3]))
print('3 last real for freq domain: ', (amp[-3:]))
print('3 first imag for freq domain: ', (phase[:3]))
print('3 last imag for freq domain: ', (phase[-3:]))

In [ ]:
print('3 first real for freq domain: ', (freq_domain_real[:3]))
print('3 last real for freq domain: ', (freq_domain_real[-3:]))
print('3 first imag for freq domain: ', (freq_domain_imag[:3]))
print('3 last imag for freq domain: ', (freq_domain_imag[-3:]))

In [ ]:
import numpy as np

In [ ]:
from dcrhino3.signal_processing.filters import FIRLSFilter

my_filter = FIRLSFilter([15, 30, 100, 150], 0.1)
my_filter.make(5000.0)
bb = my_filter.taps
aa = np.asarray([1.0,])

In [ ]:
w, h = signal.freqs(bb, aa)
plt.semilogx(w/(2*np.pi), 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Amplitude [dB]')
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()

In [ ]:
samples_from_center = 25

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharey=True, squeeze=True)
# b_filt, a_filt = signal.butter(5, np.array([30/5000, 100/5000]), btype='bandpass')

for r in range(1300, 2500+500, 500):
    amp, phase = amplitude_and_phase(pipe, Rock(1700, r), f)
    ampm, phase = amp, phase
    array = inverse_transform(amp, phase)
#     array = array[8000:10250]
#     array = signal.resample(array, 1000)
#     array = signal.resample(array, 10000)
#     array = signal.filtfilt(bb, aa, array)
    array = array[(int(N/2))-samples_from_center:(int(N/2))+samples_from_center]
    
#     peaks = signal.find_peaks(array, distance=10)[0]
#     left, right = peaks[array[peaks].argsort()[-2:]]
#     peaks = signal.find_peaks(0-array, distance=10)[0]
#     center = peaks[np.argmin(array[peaks])]
    
    ax1.plot(array, label='alpha = 1700; rho = {}'.format(r), lw=1)
#     ax1.scatter(np.r_[right, left, center], array[np.r_[right, left, center]])
    
for a in range(1000, 3500+500, 500):
    amp, phase = amplitude_and_phase(pipe, Rock(a, 1500), f)
    amp, phase = amp, phase
    array = inverse_transform(amp, phase)
#     array = array[8000:10250]
#     array = signal.resample(array, 1000)
#     array = signal.resample(array, 10000)
#     array = signal.filtfilt(bb, aa, array)
    array = array[(int(N/2))-samples_from_center:(int(N/2))+samples_from_center]
    
#     peaks = signal.find_peaks(array, distance=10)[0]
#     left, right = peaks[array[peaks].argsort()[-2:]]
#     peaks = signal.find_peaks(0-array, distance=10)[0]
#     center = peaks[np.argmin(array[peaks])]
    
    ax2.plot(array, label='alpha = {}; rho = 1500'.format(a), lw=1)
#     ax2.scatter(np.r_[right, left, center], array[np.r_[right, left, center]])
    
for ax in [ax1, ax2]:
    ax.legend()
    ax.grid()
    
fig.subplots_adjust(wspace=0)
fig.set_size_inches(25,10)

In [ ]:
array.shape

In [ ]:
np.pad(array, 200, mode='constant', constant_values=0).shape

In [ ]:
1 / f.max()

In [ ]:
np.pad([1,1,1], [6, 0], mode='constant', constant_values=0)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, sharey=True, squeeze=True)
b_filt, a_filt = signal.butter(3, np.array([30/5000, 100/5000]), btype='bandpass')

for r in range(1300, 2500+500, 500):
    
    amp, phase = amplitude_and_phase(pipe, Rock(1000, r), f)
    ampm, phase = amp, phase
    array = inverse_transform(amp, phase, n=20000)
    array = array[9950:10050]
    array = np.pad(array, 200, mode='constant', constant_values=0)
#     array = array[8000:10250]
#     array = signal.resample(array, 1000)
#     array = signal.resample(array, 10000)
    array = signal.filtfilt(b_filt, a_filt, array)
    
    ac_array = signal.convolve(array, array, mode='same', method='direct')

    ac_array = signal.filtfilt(b_filt, a_filt, ac_array)
    ac_array = np.pad(ac_array, [69, 0], mode='constant', constant_values=0)[:500]
    ac_array = (ac_array * (max(array) / max(ac_array)))
#     peaks = signal.find_peaks(array, distance=10)[0]
#     left, right = peaks[array[peaks].argsort()[-2:]]
#     peaks = signal.find_peaks(0-array, distance=10)[0]
#     center = peaks[np.argmin(array[peaks])]
    jazzbass = array + ac_array
    jazzbass = (jazzbass * -1) / 0.0049954532781795284
    ax1.plot((jazzbass), label='alpha = 1000; rho = {}'.format(r), lw=1)
#     ax1.scatter(np.r_[right, left, center], array[np.r_[right, left, center]])
    
for a in range(1000, 3500+500, 500):
    amp, phase = amplitude_and_phase(pipe, Rock(a, 1300), f)
    amp, phase = amp, phase
    array = inverse_transform(amp, phase, n=20000)
    array = array[9950:10050]
    array = np.pad(array, 200, mode='constant', constant_values=0)
#     array = array[8000:10250]
#     array = signal.resample(array, 1000)
#     array = signal.resample(array, 10000)
    array = signal.filtfilt(b_filt, a_filt, array)
    
    ac_array = signal.convolve(array, array, mode='same', method='direct')

    ac_array = signal.filtfilt(b_filt, a_filt, ac_array)
    ac_array = np.pad(ac_array, [69, 0], mode='constant', constant_values=0)[:500]
    ac_array = (ac_array * (max(array) / max(ac_array))) 
#     peaks = signal.find_peaks(array, distance=10)[0]
#     left, right = peaks[array[peaks].argsort()[-2:]]
#     peaks = signal.find_peaks(0-array, distance=10)[0]
#     center = peaks[np.argmin(array[peaks])]
    jazzbass = array + ac_array
    jazzbass = (jazzbass * -1) / 0.0049954532781795284
    ax2.plot((jazzbass), label='alpha = {}; rho = 1300'.format(a), lw=1)
#     ax2.scatter(np.r_[right, left, center], array[np.r_[right, left, center]])
    
for ax in [ax1, ax2]:
    ax.legend()
    ax.grid()
    
fig.subplots_adjust(wspace=0)
fig.set_size_inches(25,10)

In [ ]:
alpha=np.arange(1000, 3500 + 25, 25)
rho=np.arange(1300, 2500 + 25, 25)

In [ ]:
pipe.rho

In [ ]:
plt.plot(data[9950:10050])

In [ ]:
plt.plot(data[9950:10050])

In [ ]:
import itertools

outputs = dict()
order = 1

b_filt, a_filt = signal.butter(3, np.array([30/5000, 100/5000]), btype='bandpass')

for _i, (r, a) in enumerate(itertools.product(rho, alpha)):
    if _i % 1000 == 0:
        print('rho {:.3f}, alpha {:.3f}'.format(r, a))
    
    amp, phase = amplitude_and_phase(pipe, Rock(a, r), f)
    amp, phase = amp**order, phase*order
    data = inverse_transform(amp, phase, n=20000)
    
    array = data[9950:10050]
    array = np.pad(array, 200, mode='constant', constant_values=0)
    array = signal.filtfilt(b_filt, a_filt, array)
    
    ac_array = signal.convolve(array, array, mode='same', method='direct')

    ac_array = signal.filtfilt(b_filt, a_filt, ac_array)
    
    ac_array = np.pad(ac_array, [69, 0], mode='constant', constant_values=0)[:500]
    ac_array = (ac_array * (max(array) / max(ac_array))) 
    
    jazzbass = array + ac_array
    jazzbass = (jazzbass * -1) / 0.0049954532781795284
    
    peaks = signal.find_peaks(jazzbass, distance=10)[0]
    right, left, center = peaks[jazzbass[peaks].argsort()[-3:]]
   
    outputs.setdefault('pipe_outer_radius', list()).append(pipe.Ro)
    outputs.setdefault('pipe_inner_radius', list()).append(pipe.Ri)
    outputs.setdefault('pipe_effective_bit_radius', list()).append(pipe.Rb)
    outputs.setdefault('pipe_rho', list()).append(pipe.rho)
    outputs.setdefault('pipe_alpha', list()).append(pipe.alpha)
    
    outputs.setdefault('rock_rho', list()).append(r)
    outputs.setdefault('rock_alpha', list()).append(a)

    outputs.setdefault('amplitude_and_phase_order', list()).append(order)

#     outputs.setdefault('rock_amplitude_left', list()).append(abs(jazzbass[left]))
#     outputs.setdefault('rock_amplitude_right', list()).append(abs(jazzbass[right]))
#     outputs.setdefault('rock_amplitude_center', list()).append(abs(jazzbass[center]))
#     outputs.setdefault('rock_jazz_bass', list()).append(abs(jazzbass[right]) - abs(jazzbass[center]) - abs(jazzbass[left]))
    outputs.setdefault('rock_modulus', list()).append((r)*(a**2))
    
    outputs.setdefault('modeled_wavelet', list()).append(jazzbass.tolist())
    
outputs['rock_modulus'] = np.array(outputs['rock_modulus'])*1.2e-9

In [ ]:
frequency_resolution

In [ ]:
delta_t = 0.0001

In [ ]:
data[9950:10050]

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
dataframe = pd.DataFrame(outputs)

In [ ]:
sns.scatterplot('rock_modulus', 'rock_jazz_bass', data=data)

In [ ]:
import os

In [ ]:
os.makedirs('/data/datacloud/theoretical_rhino/', exist_ok=True)
data.to_csv('/data/datacloud/theoretical_rhino/modeled_wavelets.csv', index=False)

In [ ]:
peaks = signal.find_peaks(jazzbass, distance=10)[0]
right, left, center = peaks[jazzbass[peaks].argsort()[-3:]]

plt.plot(jazzbass)
plt.scatter(center, jazzbass[center])
plt.scatter(left, jazzbass[left])
plt.scatter(right, jazzbass[right])
plt.grid()
plt.gcf().set_size_inches(25, 6)

In [ ]:
import pandas as pd

In [ ]:
pd.Series(phases).describe()

In [ ]:
x = np.linspace(np.array(phases).min(), np.array(phases).max(), 100)

poly = np.poly1d(np.polyfit(phases, modulus, 4))

poly

In [ ]:
np.poly1d([  0.5849666 ,   6.62621535,  27.40729208,  52.01668517,  46.29158229])

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(phases)

In [ ]:
['{:.3f}'.format(c) for c in poly]

In [ ]:
plt.style.use('seaborn-dark')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(25,5))

for ax, curve, y_label in zip(axes.ravel(), [modulus, rhos, alphas], ['Modulus (GPa)', 'Density (g/cm³)', 'Velocity (m/s)']):
    x = np.linspace(np.array(phases).min(), np.array(phases).max(), 100)
    poly = np.poly1d(np.polyfit(phases, curve, 4))
    ax.scatter(phases, curve, s=.5)
    ax.plot(x, poly(x), c='r')
    ax.set_ylabel(y_label)
    ax.set_xlabel('JazzBass')

    ax.grid()

In [ ]:
import inflect
p = inflect.engine()

from obspy import read, Trace, Stream, UTCDateTime
from obspy.core import AttribDict
from obspy.io.segy.segy import SEGYTraceHeader, SEGYBinaryFileHeader
from obspy.io.segy.core import _read_segy
import numpy as np
import sys
import itertools

order = 1
resampled = True

for order, resampled  in zip([1, 1, 2, 2, 3, 3, 4, 4], [True, False] * 4):
    stream = Stream()
    
    resampled_string = 'RESAMPLED' if resampled else 'RAW'

    for _i, (r, a) in enumerate(itertools.product(rho, alpha)):
        if _i % 1000 == 0:
            print('rho {:.3f}, alpha {:.3f}'.format(r, a))
        amp, phase = amplitude_and_phase(pipe, Rock(a, r), f)
        amp, phase = amp**order, phase*order
        data = inverse_transform(amp, phase, n=20000)
        data = data[9950:10050]
        if resampled:
            data = signal.resample(data, 1000)
        # Create some random data.
        data = np.require(data, dtype=np.float32)
        trace = Trace(data=data)

        # Attributes in trace.stats will overwrite everything in
        # trace.stats.segy.trace_header
        trace.stats.sampling_rate = 300
    #     trace.stats.delta = 3

        # SEGY does not support microsecond precision! Any microseconds will
        # be discarded.
        trace.stats.starttime = UTCDateTime(2019, 4,30,0,0,0)

        # If you want to set some additional attributes in the trace header,
        # add one and only set the attributes you want to be set. Otherwise the
        # header will be created for you with default values.
        if not hasattr(trace.stats, 'segy.trace_header'):
            trace.stats.segy = {}
        trace.stats.segy.trace_header = SEGYTraceHeader(unpack_headers=False)
        trace.stats.segy.trace_header.trace_sequence_number_within_line = _i + 1
        trace.stats.segy.trace_header.rho = int(r)
        trace.stats.segy.trace_header.alpha = int(a)

        # Add trace to stream
        stream.append(trace)

    # A SEGY file has file wide headers. This can be attached to the stream
    # object.  If these are not set, they will be autocreated with default
    # values.
    stream.stats = AttribDict()
    stream.stats.textual_file_header = 'Textual Header!'
    stream.stats.binary_file_header = SEGYBinaryFileHeader()
    stream.stats.binary_file_header.trace_sorting_code = 5

    print("Stream object before writing...")
    print(stream)

    stream.write("rhino_theoretical_wavelets_{}order_{}.sgy".format(p.ordinal(order), resampled_string), format="SEGY", data_encoding=1,
                 byteorder=sys.byteorder)
    print("Stream object after writing. Will have some segy attributes...")
    print(stream)

# print("Reading using obspy.io.segy...")
# st1 = _read_segy("rhino_theoretical_wavelets.sgy", unpack_trace_headers=False)
# print(st1)